# Análise das Intents

Este jupyter-notebook vai auxiliar na análise de detecção de intenções de seu chatbot.

### Configurando e Imports

In [1]:
from IPython.display import IFrame

import rasa_nlu
print("rasa_nlu: {}".format(rasa_nlu.__version__))

rasa_nlu: 0.15.0


/usr/local/lib/python3.6/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.2) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


### Treinando o modelo do Rasa NLU

* Para avaliar o bot o primeiro passo é treiner o seu chatbot. Mas não é necessário treinar a parte de conversão completa (rasa_core) apenas a parte de interpretação de mensagens (rasa_nlu).

* O comando `train-nlu` do Makefile executa o treinamento apenas do `rasa_nlu`

In [2]:
!make train-nlu -C $BOT_DIR_PATH

make: Entering directory '/work/bot'
python3 -m rasa_nlu.train -c nlu_config.yml --fixed_model_name current \
  --data data/intents/ -o models --project nlu --verbose
/usr/local/lib/python3.6/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.2) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
/usr/local/lib/python3.6/runpy.py:125: RuntimeWarning: 'rasa_nlu.train' found in sys.modules after import of package 'rasa_nlu', but prior to execution of 'rasa_nlu.train'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2019-05-03 21:20:31 INFO     rasa_nlu.training_data.loading  - Training data format of data/intents/aleatorio.md is md
2019-05-03 21:20:31 INFO     rasa_nlu.training_data.training_data  - Training data stats: 
	- intent examples: 197 (24 distinct intents)
	- Found intents: 'historia', 'de_onde_voce_eh', 'triste', 'filhos', 'hobby', 'signo', 'como_estou', 'piada', 'playlist', 'time', 'genero', 'r

### Métodos de avaliação do chatbot

* O Rasa fornece vários métodos de avaliação e validação das `intents`, para verificar como utiliza-los, cada método fornece um log, imagem, gráfico ou arquivo com dados relevantes para interpretação do chatbot.

* O comando `!python -m rasa_nlu.test` é a base para a avaliação do chatbot.

* Na célula abaixo a flag `-h` foi utilizada para mostrar as funções e a forma de uso de cada uma delas, mude seus valores e flags para ter as informações desejadas na sua análise.

* Atualmente o Rasa possui 2 modos, `evaluation` e `crossvalidation` que tem seções decicadas a eles neste jupyter notebook.

In [3]:
!python -m rasa_nlu.test -h

/usr/local/lib/python3.6/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.2) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
/usr/local/lib/python3.6/runpy.py:125: RuntimeWarning: 'rasa_nlu.test' found in sys.modules after import of package 'rasa_nlu', but prior to execution of 'rasa_nlu.test'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
usage: test.py [-h] [--debug] [-v] -d DATA [--mode MODE] [-c CONFIG]
               [-m MODEL] [-f FOLDS] [--report [REPORT]]
               [--successes [SUCCESSES]] [--errors ERRORS]
               [--histogram HISTOGRAM] [--confmat CONFMAT]

evaluate a Rasa NLU pipeline with cross validation or on external data

optional arguments:
  -h, --help            show this help message and exit
  --debug               Print lots of debugging statements. Sets logging level
                        to DEBUG
  -v, --verbose         Be verbose. Sets logging level to INFO

### Evaluation

* O comando abaixo gera informações relevates para a validação das `intents` são elas:
    * Matriz de confuzão
    * Histograma
    * Erros de intents

In [4]:
!python -m rasa_nlu.test -d $BOT_INTENTS_PATH -m $BOT_MODELS_NLU_PATH --mode evaluation

/usr/local/lib/python3.6/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.2) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
/usr/local/lib/python3.6/runpy.py:125: RuntimeWarning: 'rasa_nlu.test' found in sys.modules after import of package 'rasa_nlu', but prior to execution of 'rasa_nlu.test'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2019-05-03 21:20:46.960890: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2019-05-03 21:20:46.979691: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2905000000 Hz
2019-05-03 21:20:46.979956: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x565180f44560 executing computations on platform Host. Devices:
2019-05-03 21:20:46.979989: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <un

#### Matriz de Confuzão
* A matriz de confuzão mostra a correlação entre as intents.
* A diagonal principal tem forte correlação pois mostra a relação de uma intent **com ela mesma**
* O ideal é que não haja **nenhum valor** diferente de **0 fora da diagonal principal**.

In [5]:
IFrame(src='./confmat.png', width=900, height=700)

#### Erros
* O arquivo `erros,json` mostra os erros encontrados. Este arquivo lista os mesmos erros mostrados na **matriz de confuzão**, então caso ela fique muito grande, não se preocupe, pois você pode apenas procurar os erros no arquivo gerado.
* Os erros mostrados são textos repetidos nos exemplos de diferrentes `intents`.
* Caso o arquivo não seja gerado significa que não foram encontrados erros.

In [6]:
%cat errors.json

[
    {
        "text": "e o c#",
        "intent": "linguagens",
        "intent_prediction": {
            "name": "",
            "confidence": 0.0
        }
    },
    {
        "text": "como vai",
        "intent": "cumprimentar",
        "intent_prediction": {
            "name": "tudo_bem",
            "confidence": 0.835846483707428
        }
    },
    {
        "text": "tudo bom",
        "intent": "cumprimentar",
        "intent_prediction": {
            "name": "tudo_bem",
            "confidence": 0.8136337995529175
        }
    }
]

#### Histograma

* O histograma contém a distribuição da predições das `intents`

In [7]:
IFrame(src='./hist.png', width=900, height=700)

### Crossvalidation

In [8]:
!python -m rasa_nlu.test -d $BOT_INTENTS_PATH -c $BOT_NLU_CONFIG_PATH --mode crossvalidation --folds 2 --report

/usr/local/lib/python3.6/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.2) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
/usr/local/lib/python3.6/runpy.py:125: RuntimeWarning: 'rasa_nlu.test' found in sys.modules after import of package 'rasa_nlu', but prior to execution of 'rasa_nlu.test'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2019-05-03 21:20:56 INFO     rasa_nlu.training_data.loading  - Training data format of ../../bot/data/intents/aleatorio.md is md
2019-05-03 21:20:56 INFO     rasa_nlu.training_data.training_data  - Training data stats: 
	- intent examples: 197 (24 distinct intents)
	- Found intents: 'linguagens', 'triste', 'cor', 'hobby', 'risada', 'bff', 'genero', 'star_wars', 'religiao', 'filhos', 'como_estou', 'me', 'signo', 'historia', 'relationship', 'filme', 'playlist', 'time', 'license', 'piada', 'comida', 'onde_voce_mora', 'esporte', 'de_onde_voce_eh'
	- entity examples

2019-05-03 21:21:00.706185: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2019-05-03 21:21:00.726392: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2905000000 Hz
2019-05-03 21:21:00.726674: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x555b9ee46db0 executing computations on platform Host. Devices:
2019-05-03 21:21:00.726702: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
2019-05-03 21:21:00 INFO     rasa_nlu.classifiers.embedding_intent_classifier  - Accuracy is updated every 10 epochs
Epochs: 100%|██████████| 300/300 [00:05<00:00, 59.85it/s, loss=0.186, acc=1.000]
2019-05-03 21:21:05 INFO     rasa_nlu.classifiers.embedding_intent_classifier  - Finished training embedding classifier, loss=0.186, train accuracy=1.000
2019-05-03 21:21:05 INFO     rasa_nlu.model  - Finished training compone

## Referências:

O Rasa está em constante evolução, alguns links úteis para a construção deste jupyter-notebook e para a análise das `intents` são:

* [Evaluation](https://rasa.com/docs/nlu/evaluation/)